## hard voting

In [1]:
import os
from os import path as osp
import pandas as pd
#import numpy as np
from tqdm import tqdm

In [2]:
CLASSES = [ 
    'Background', 'General trash', 'Paper', 'Paper pack', 'Metal', 'Glass',
    'Plastic', 'Styrofoam', 'Plastic bag', 'Battery', 'Clothing'
]

In [3]:
base_dir = '../../submission/ensemble_V3/'
target = []
target.append('adapter_fold1_val784')# best model
target.append('adapter_fold0') 
target.append('adapter_fold2_val7965')

output_name = f'{target[0]}_ensemble'

dfs = []
for file_name in target:
    dfs.append(pd.read_csv(osp.join(base_dir, file_name+'.csv')))

submission = pd.DataFrame()
submission['image_id'] = dfs[0]['image_id']

In [4]:
PredictionString = []

for idx in tqdm(range(len(dfs[0]))):
    l_result = []
    for df in dfs:
        l_result.append(df['PredictionString'][idx].split(' '))
        
    result = ''
    for i in range(len(l_result[0])):
        pixel_count = {str(i):0 for i in range(11)}
        
        best_pixel = l_result[0][i]
        for t_result in l_result:
            pixel_count[t_result[i]] += 1
        voted_pixel = [key for key, value in pixel_count.items() if value == max(pixel_count.values())]
        
        # voted_pixel이 2개 이상인 경우
        if len(voted_pixel) > 1:
            # best_pixel이 뽑은 pixel이 있는 경우
            if best_pixel in pixel_count:
                result += best_pixel + ' '
        # 1개만 뽑힌 경우 (model이 4개 이하일 경우만)
        else:
            result += voted_pixel[0] + ' '
            
    ensemble_result = result[:-1]
    PredictionString.append(ensemble_result)

  1%|          | 5/819 [00:02<05:51,  2.32it/s]


KeyboardInterrupt: 

In [ ]:
submission['PredictionString'] = PredictionString
submission.to_csv(base_dir + f'{output_name}.csv', index=False)